# HKU QIDS 2023 Quantitative Investment Competition: Model

## Init Config

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from qids_package.qids import *
import warnings
from submit import submit

from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")

In [7]:
seed = 257248
stock_num = 54
day_num_total = 1000
day_num = 1000 - 2
test_day_num = 700
timeslot_num = 50

In [19]:
def std(train, valid, test=None):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    valid = scaler.transform(valid)
    if test is not None:
        test = scaler.transform(test)
    return train, valid, test

In [74]:
def calc_corr(df1, df2):
    return np.corrcoef(df1, df2)[0][1]

In [82]:
def evaluate(model, train, valid, test, train_y, valid_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = model.predict(train)
        model_valid_y = model.predict(valid)
        pred = model.predict(test)
        acr_train = model.score(train, train_y)
        acr_valid = model.score(valid, valid_y)
        print(calc_corr(train_y, model_train_y))
        print(calc_corr(valid_y, model_valid_y))
        if return_pred:
            return pred
    #     cm = confusion_matrix(train_y, model_train_y)
    #     tn, fp, fn, tp = cm.ravel()
    #     print(f"Train AUROC: {roc_auc_score(train_y, model_train_y):.4f}; FP: {fp:>4d}; FN: {fn:>4d}")
    #     cm = confusion_matrix(valid_y, model_valid_y)
    #     tn, fp, fn, tp = cm.ravel()
    #     print(f"Valid AUROC: {roc_auc_score(valid_y, model_valid_y):.4f}; FP: {fp:>4d}; FN: {fn:>4d}")
    #     print()
    # elif version == 3:
    #     pred_train = model.predict(train)
    #     pred_valid = model.predict(valid)
    #     pred = model.predict(test)
    #     print(f"Train RMSE: {mse(train_y, pred_train, squared=False):.4f}")
    #     print(f"Valid RMSE: {mse(valid_y, pred_valid, squared=False):.4f}")
    # else:
    #     model_train_y = model.predict_proba(train)[:,1]
    #     model_valid_y = model.predict_proba(valid)[:,1]
    #     pred = model.predict_proba(test)[:,1]
    #     acr_train = model.score(train, train_y)
    #     acr_valid = model.score(valid, valid_y)
    #     print(f"Train Accuracy: {acr_train:.4f}; Validation Accuracy: {acr_valid:.4f}")
    #     auc_train = roc_auc_score(train_y, model_train_y)
    #     auc_valid = roc_auc_score(valid_y, model_valid_y)
    #     print(f"Train AUROC: {auc_train:.4f}")
    #     print(f"Valid AUROC: {auc_valid:.4f}")
    #     print()
    # if plot_auc:
    #     try:
    #         fpr1, tpr1, thresholds1 = roc_curve(train_y, model_train_y)
    #         fpr2, tpr2, thresholds2 = roc_curve(valid_y, model_valid_y)
    #         plt.plot([0,1], 'k--')
    #         plt.plot(fpr1, tpr1, label= "Train")
    #         plt.plot(fpr2, tpr2, label= "Valid")
    #         plt.legend()
    #         plt.xlabel("FPR")
    #         plt.ylabel("TPR")
    #         plt.title('AUROC Curve')
    #         plt.show()
    #     except:
    #         pass
    # if return_pred and return_auc and version == 1:
    #     return pred, auc_train, auc_valid
    # elif return_pred:
    #     return pred
    # else:
    #     return

## Load Data

In [83]:
write_path = "../data/"

train_path = write_path + "train.csv"
valid_path = write_path + "valid.csv"
test_path = write_path + "test.csv"

train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)
test = pd.read_csv(test_path)

In [84]:
train_y = train["return"]
train = train.drop(columns=["return", "date_time"])

valid_y = valid["return"]
valid = valid.drop(columns=["return", "date_time"])

test = test.drop(columns=["date_time"])

## Normalization

In [21]:
# train, valid, test = std(train.iloc[:, 3:-1], valid.iloc[:, 3:-1], test.iloc[:, 3:])

## Model

### Linear Regression

In [86]:
model = LinearRegression()
pred = evaluate(model, train, valid, test, train_y, valid_y)

0.08990825442268606
0.08413533119889033


In [87]:
pred

array([-0.00098012, -0.00070065, -0.00069922, ..., -0.00972251,
       -0.00787746, -0.00710269])

## Submission

In [88]:
submit(pred)